## Using XGBoost in pipelines

In [3]:
import xgboost as xgb
import pandas as pd
import numpy as np

### Encoding categorical columns I: LabelEncoder

In [4]:
df = pd.read_csv('chapter4-data.csv')

In [5]:
df.shape

(1460, 22)

In [6]:

from sklearn.preprocessing import LabelEncoder


df.LotFrontage = df.LotFrontage.fillna(0)


categorical_mask = (df.dtypes == object)


categorical_columns = df.columns[categorical_mask].tolist()


print(df[categorical_columns].head())


le = LabelEncoder()


df[categorical_columns] = df[categorical_columns].apply(lambda x: le.fit_transform(x))


print(df[categorical_columns].head())

  MSZoning PavedDrive Neighborhood BldgType HouseStyle
0       RL          Y      CollgCr     1Fam     2Story
1       RL          Y      Veenker     1Fam     1Story
2       RL          Y      CollgCr     1Fam     2Story
3       RL          Y      Crawfor     1Fam     2Story
4       RL          Y      NoRidge     1Fam     2Story
   MSZoning  PavedDrive  Neighborhood  BldgType  HouseStyle
0         3           2             5         0           5
1         3           2            24         0           2
2         3           2             5         0           5
3         3           2             6         0           5
4         3           2            15         0           5


### Encoding categorical columns II: OneHotEncoder

In [7]:

from sklearn.preprocessing import OneHotEncoder


ohe = OneHotEncoder(sparse=False)


df_encoded = ohe.fit_transform(df)


print(df_encoded[:5, :])


print(df.shape)


print(df_encoded.shape)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(1460, 22)
(1460, 4829)


### Encoding categorical columns III: DictVectorizer

In [8]:

from sklearn.feature_extraction import DictVectorizer


df_dict = df.to_dict("records")


dv = DictVectorizer(sparse=False)


df_encoded = dv.fit_transform(df_dict)


print(df_encoded[:5,:])


print(dv.vocabulary_)

[[3.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 2.000e+00 5.480e+02
  1.710e+03 1.000e+00 5.000e+00 8.450e+03 6.500e+01 6.000e+01 3.000e+00
  5.000e+00 5.000e+00 7.000e+00 2.000e+00 0.000e+00 2.085e+05 0.000e+00
  2.003e+03]
 [3.000e+00 0.000e+00 0.000e+00 1.000e+00 1.000e+00 2.000e+00 4.600e+02
  1.262e+03 0.000e+00 2.000e+00 9.600e+03 8.000e+01 2.000e+01 3.000e+00
  2.400e+01 8.000e+00 6.000e+00 2.000e+00 0.000e+00 1.815e+05 1.000e+00
  1.976e+03]
 [3.000e+00 0.000e+00 1.000e+00 0.000e+00 1.000e+00 2.000e+00 6.080e+02
  1.786e+03 1.000e+00 5.000e+00 1.125e+04 6.800e+01 6.000e+01 3.000e+00
  5.000e+00 5.000e+00 7.000e+00 2.000e+00 1.000e+00 2.235e+05 2.000e+00
  2.001e+03]
 [3.000e+00 0.000e+00 1.000e+00 0.000e+00 1.000e+00 1.000e+00 6.420e+02
  1.717e+03 0.000e+00 5.000e+00 9.550e+03 6.000e+01 7.000e+01 3.000e+00
  6.000e+00 5.000e+00 7.000e+00 2.000e+00 1.000e+00 1.400e+05 3.000e+00
  1.915e+03]
 [4.000e+00 0.000e+00 1.000e+00 0.000e+00 1.000e+00 2.000e+00 8.360e+02
  2.198e+03 

### Preprocessing within a pipeline

In [9]:
X = df

In [11]:
df.head()

,Unnamed: 0,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,Remodeled,GrLivArea,BsmtFullBath,...,HalfBath,BedroomAbvGr,Fireplaces,GarageArea,MSZoning,PavedDrive,Neighborhood,BldgType,HouseStyle,SalePrice
0,0,60,65.0,8450,7,5,2003,0,1710,1,...,1,3,0,548,3,2,5,0,5,208500
1,1,20,80.0,9600,6,8,1976,0,1262,0,...,0,3,1,460,3,2,24,0,2,181500
2,2,60,68.0,11250,7,5,2001,1,1786,1,...,1,3,1,608,3,2,5,0,5,223500
3,3,70,60.0,9550,7,5,1915,1,1717,1,...,0,3,1,642,3,2,6,0,5,140000
4,4,60,84.0,14260,8,5,2000,0,2198,1,...,1,4,1,836,3,2,15,0,5,250000


In [12]:
y = df.SalePrice

In [13]:

from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline


X.LotFrontage = X.LotFrontage.fillna(0)


steps = [("ohe_onestep", DictVectorizer(sparse=False)),
         ("xgb_model", xgb.XGBRegressor())]


xgb_pipeline = Pipeline(steps)


xgb_pipeline.fit(X.to_dict("records"), y)

Pipeline(steps=[('ohe_onestep', DictVectorizer(sparse=False)),
                ('xgb_model',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

### Cross-validating your XGBoost model

In [ ]:

from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score


X.LotFrontage = X.LotFrontage.fillna(0)


steps = [("ohe_onestep", DictVectorizer(sparse=False)),
         ("xgb_model", xgb.XGBRegressor(max_depth=2, objective="reg:linear"))]


xgb_pipeline = Pipeline(steps)


cross_val_scores = cross_val_score(xgb_pipeline, X.to_dict("records"), y, cv=10, scoring="neg_mean_squared_error")


print("10-fold RMSE: ", np.mean(np.sqrt(np.abs(cross_val_scores))))